# Search Engine

This lab is about starting from scratch, and hence having to structure the code yourself. A specification for the program you are to implement is given below - do pay attention as misunderstandings may cost you marks! Being precise is a necessary skill for a programmer. To give an executive summary, you are to code a search engine for recipes. A data set has been provided. The search engine is to be pretty basic, returning all recipes that contain all of the provided keywords. However, the user can choose from a number of orderings depending on their food preferences, which you need to support.

## Specification

The system must provide a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. Must `print` just their title, one per line. Must be less than `count` if the search is specific enough that less than `count` recipes match.

As an aside, do not worry about memory usage. If duplicating some data can make your code faster/neater then feel free.



### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. Your code will need to cope with this.

You will probably want to explore the data before starting, so you have an idea of what your code has to deal with.

Data set came from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though note it has been cleaned it up, by deleting duplicates and removing the really dodgy entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is to be ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the `re` module, but there is no need to do that here (`re` can be quite fiddly). For matching words your tokenisation must follow the following steps:
1. Convert all punctuation and digits into spaces. For punctuation use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits). You may find [`translate()`](https://docs.python.org/3/library/stdtypes.html#str.translate) interesting!
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search (above) or ordering (below) it's only a match if you match an entire token. There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The auto marker will be checking the above is followed! When doing a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$)

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet. Try not to overthink the optimisation of $n$, as it's really quite simple to do!

To clarify the use of the ordering string, to get something healthy that contains cheese you might call `search('cheese', 'healthy')`. In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry in the recipe (e.g. no `ingredients` are provided) to simply mean that entry can't match any search words (because it has none!), but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.



### Extra

You may find the [_inverted index_](https://en.wikipedia.org/wiki/Inverted_index) interesting. It's a data structure used by search engines. For each word a user may search for this contains a list of all documents (recipes) that contain the word. This may take a little effort to understand, but the resulting code will be both faster and neater.

In [1]:
import pandas as pd
import numpy as np
import string
import unicodedata
df_recipes = pd.read_json(r'recipes.json')

In [2]:
def return_entry_terms(df):
    """takes in a dataframe and returns a list of all clean terms"""

    # empty string to collect text entries
    ttext = []

    # create a single string of all collated text data
    for i in ['title','categories','ingredients','directions']:
        for j in range(df.shape[0]):         # add full range once working   
            entry = df[i].iloc[j]
            
            #concatenate list to strings
            if type(entry) == list:
                
                for z in entry:
                    ttext.append(z)
            
            #remove nans
            elif pd.isna(entry):
                pass
            else:
                ttext.append(entry)
    
    text = ' '.join(ttext)

    s_text = clean_string_to_list(text)

    s_set = set(s_text)

    ret = dict(zip(s_set,range(len(s_set))))

    return ret

In [3]:
def clean_string_to_list(text):
    """takes a string and returns a list of cleaned terms"""
    # clean \n and \xa0
    text1 = unicodedata.normalize("NFKD", text)
    ttext = text1.replace("\n",' ').replace("\\",' ').replace("—",' ').replace("–",' ').replace("’",' ').replace("•",' ')
    
    # create translation tables for removing digits and punctuation 
    mytable = ttext.maketrans(string.digits,' ' * len(string.digits))
    mytable2 = ttext.maketrans(string.punctuation,' ' * len(string.punctuation))    
    # remove digits and punctuation
    ttext = ttext.translate(mytable).translate(mytable2)
    input()
    # split into list
    o_text = ttext.split(' ')
    
    ret = []
    #remove empty terms
    for i in o_text:
        if len(i) >= 3:
            ret.append(i.lower())
            
    return ret

In [ ]:
# generate the cleaned list of terms
clean_dict = return_entry_terms(df_recipes)

In [ ]:
def create_inv_index_array(clean_dict,df):
    """Creates a 3d indexing array to search recipes against"""
    
    ret = np.zeros((df.shape[0],len(clean_dict),4),dtype=np.int8) 
    
    col_dic = {'title' : 0 ,'categories' : 1 ,'ingredients' : 2 ,'directions': 3}
    
    for i in col_dic.keys():
        for j in range(df.shape[0]):
            
            entry = df[i].iloc[j]
            
            #concatenate list to strings
            if type(entry) == list:
                text = ' '.join(entry)     
            elif pd.isna(entry):
                continue                             
            else:
                text = entry
            
            s_text = clean_string_to_list(text)
            
            for k in s_text:
                ret[j, clean_dict[k] ,col_dic[i]] += 1
        
    return ret

In [ ]:
%%time
#create the array index

arr_ix = create_inv_index_array(clean_dict,df_recipes)

In [ ]:
def search(query, ordering = 'normal', count = 10):
    
    # find indices of the search term in the index array
    query = clean_string_to_list(query)
    if query == []:
        return
    elif len(query) == 1:
        query = query[0] #for single term
    else:
        pass
    
    global clean_dict
    global arr_ix
    global df_recipes
    
    # indices of search query in the terms available 
    try:
        if type(query) == str:
            j_ix = clean_dict[query.lower()]
        else:
            j_ix = []
            for i in query:
                j_i = clean_dict[i.lower()]
                j_ix.append(j_i)
            tt = arr_ix[:,j_ix,:].sum(axis=2)
            ttx = tt[:,0]
            for i in range(1,len(j_ix)):
                ttx *= tt[:,i]
            match_ix = np.where(ttx>0)[0]
            # match_ix is the indices of the matching recipes

    except:
        return
    
    # get indices of matches to the search term
    if type(query) == str:
        match_ix = np.argwhere(arr_ix[:,j_ix,:].sum(axis=1) >0)[:,0]    
    #return a pandas series of matching recipe names 
        
    recipes = df_recipes['title'].iloc[match_ix]

    if ordering == 'normal':
        
        #get ratings and convert nans to 0
        
        ratings = df_recipes['rating'].iloc[match_ix]
        ratings.fillna(0)
        
        if type(query) == str:         
            scores = 8 * arr_ix[match_ix,j_ix,0] + 4 * arr_ix[match_ix,j_ix,1] + 2 * arr_ix[match_ix,j_ix,2] + arr_ix[match_ix,j_ix,3] + ratings.to_numpy()
        else:
            scores = ratings.to_numpy()
            for i in range(len(j_ix)):
                scores += (8 * arr_ix[match_ix,j_ix[i],0] + 4 * arr_ix[match_ix,j_ix[i],1] + 2 * arr_ix[match_ix,j_ix[i],2] + arr_ix[match_ix,j_ix[i],3])
        
            
                           
        recipes = recipes.to_frame()
    
        recipes['score'] = scores

        recipes.sort_values(by=['score'],inplace = True, ascending = False)
        
        for i in range(min(count, recipes.shape[0])):
            print(recipes['title'].iloc[i])
    
    
    elif ordering == 'simple':
        
        df_s = df_recipes.iloc[match_ix]
    
        arr_ing = np.zeros(df_s.shape[0])
        arr_dir = arr_ing.copy()
        
        for x in range(df_s.shape[0]):
            if type(df_s['ingredients'].iloc[x]) == list:
                    arr_ing[x] = len(df_s['ingredients'].iloc[x])
            if type(df_s['directions'].iloc[x]) == list:        
                    arr_dir[x] = len(df_s['directions'].iloc[x])

                    
        arr_ing = np.where(arr_ing >= 2, arr_ing, 0)
        arr_dir = np.where(arr_dir >= 2, arr_dir, 0)
        
        h_score = arr_ing * arr_dir
        
        df_s['h_score'] = h_score
        df_s = df_s[df_s['h_score'] != 0]
        df_s.sort_values(by=['h_score'],inplace = True, ascending = True)

        for i in range(min(count, df_s.shape[0])):
            print(df_s['title'].iloc[i])
        
    elif ordering == 'healthy':
        
        n=1
        df_h = df_recipes.iloc[match_ix]
        
        df_h = df_h[pd.notnull(df_h['calories'])]
        df_h = df_h[pd.notnull(df_h['fat'])]
        df_h = df_h[pd.notnull(df_h['protein'])]
        
        df_h['score'] = abs(df_h['calories'] - 510*n)/510 + 2 * abs(df_h['protein'] - 18*n)/18 + 4 * abs(df_h['fat'] - 150*n)/150
        df_h.sort_values(by=['score'],inplace = True, ascending = True)
        
        for i in range(min(count, df_h.shape[0])):
            print(df_h['title'].iloc[i])
    
        
        